[Reference](https://blog.devgenius.io/threading-in-python-d115943e7ec4)

In [1]:
import requests
import random

def get_data(idx: int) -> str:
    url = f"https://jsonplaceholder.typicode.com/todos/{idx}"
    response = requests.get(url)
    return response.json()["title"]

In [2]:
def main():
    idx = random.randint(1, 100)
    text = get_data(idx)
    print(text)
    return text

if __name__ == "__main__":
    main()

nemo perspiciatis repellat ut dolor libero commodi blanditiis omnis


In [3]:
import threading
import requests
import time
import random

results = []

def get_data(idx: int) -> str:
    url = f"https://jsonplaceholder.typicode.com/todos/{idx}"
    response = requests.get(url)
    results.append(response)
    return

def main() -> None:
    idx = random.randint(1, 100)
    start = time.time()
    for _ in range(50):
        get_data(idx)
    end = time.time()
    print(f"Time: {end-start}")
    return

if __name__ == "__main__":
    main()

Time: 10.535351753234863


In [4]:
def main():
    idx = random.randint(1, 100)
    start = time.time()
    threads = []

    for _ in range(50):
        thread = threading.Thread(target=get_data, args=(idx,))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

    end = time.time()
    print(f"Time: {end-start}")
    return

In [5]:
import threading
import time

def foo():
    print("Thread started")
    time.sleep(2)
    print("Thread finished")

# create a Thread object
t = threading.Thread(target=foo)

# start the thread
t.start()

# wait for the thread to finish
t.join()

print("Main program continues")

Thread started
Thread finished
Main program continues


In [6]:
import threading
import time

lock = threading.Lock()

# Shared resource
shared = 0


def foo():
    global shared

    thread_id = threading.get_ident()
    # Acquire the lock
    print(f"Thread {thread_id} is trying to acquire the lock")
    lock.acquire()
    print(f"Thread {thread_id} has acquired the lock")

    try:
        # Critical section
        print(f"Thread {thread_id} is inside the critical section")
        for _ in range(5):
            shared += 1
            time.sleep(0.1)
    finally:
        # Release the lock
        lock.release()
        print(f"Thread {thread_id} has released the lock")


# Create multiple threads that increment the shared variable
threads = []
for _ in range(3):
    thread = threading.Thread(target=foo)
    threads.append(thread)
    thread.start()

# Wait for all threads to finish
for thread in threads:
    thread.join()

# Print the final value of the shared variable
print("Final value of the shared variable:", shared)

Thread 137194668811840 is trying to acquire the lock
Thread 137194668811840 has acquired the lock
Thread 137194668811840 is inside the critical section
Thread 137194626848320 is trying to acquire the lockThread 137194064832064 is trying to acquire the lock

Thread 137194668811840 has released the lockThread 137194064832064 has acquired the lock
Thread 137194064832064 is inside the critical section

Thread 137194064832064 has released the lock
Thread 137194626848320 has acquired the lock
Thread 137194626848320 is inside the critical section
Thread 137194626848320 has released the lock
Final value of the shared variable: 15


In [7]:
import threading

# Shared resource
shared_variable = 0

# Create a lock and condition object
lock = threading.Lock()
condition = threading.Condition(lock)

def consumer():
    global shared_variable

    with condition:
        # Wait until the shared variable is positive
        while shared_variable <= 0:
            thread_id = threading.get_ident()
            print(f"Consumer ({thread_id}) is waiting...")
            condition.wait()

        # Consume the shared variable
        thread_id = threading.get_ident()
        print(f"Consumer ({thread_id}) consumed: {shared_variable}")
        shared_variable = 0

def producer():
    global shared_variable

    with condition:
        # Produce a positive value for the shared variable
        shared_variable = 42
        thread_id = threading.get_ident()
        print(f"Producer ({thread_id}) produced: {shared_variable}")

        # Notify the consumer that the condition is satisfied
        condition.notify()

# Create consumer and producer threads
consumer_thread = threading.Thread(target=consumer)
producer_thread = threading.Thread(target=producer)

# Start the threads
consumer_thread.start()
producer_thread.start()

# Wait for the threads to finish
consumer_thread.join()
producer_thread.join()

Consumer (137194626848320) is waiting...
Producer (137194064832064) produced: 42
Consumer (137194626848320) consumed: 42


In [8]:
import threading
import time
import random
# Create a semaphore with an initial value of 2
semaphore = threading.Semaphore(2)


def worker():
    semaphore.acquire()
    thread_id = threading.get_ident()
    print(f"Thread {thread_id} acquired the semaphore.")
    time.sleep(random.randint(1, 3))
    # Perform some task here
    print(f"Thread {thread_id} releasing the semaphore.")
    semaphore.release()


# Create worker threads
threads = []
for _ in range(5):
    t = threading.Thread(target=worker)
    threads.append(t)
    t.start()

# Wait for the threads to finish
for t in threads:
    t.join()


Thread 137194626848320 acquired the semaphore.
Thread 137194064832064 acquired the semaphore.
Thread 137194064832064 releasing the semaphore.
Thread 137194668811840 acquired the semaphore.
Thread 137194626848320 releasing the semaphore.
Thread 137194056439360 acquired the semaphore.
Thread 137194668811840 releasing the semaphore.
Thread 137194048046656 acquired the semaphore.
Thread 137194056439360 releasing the semaphore.
Thread 137194048046656 releasing the semaphore.


In [9]:
import threading
import time

# Create an event
event = threading.Event()


def worker():
    thread_id = threading.get_ident()
    print(f"Worker {thread_id} is waiting for the event.")
    event.wait()
    print(f"Worker {thread_id} has been notified of the event.")


# Create worker threads
threads = []
for _ in range(3):
    t = threading.Thread(target=worker)
    threads.append(t)
    t.start()

# Sleep for some time
time.sleep(2)

# Set the event
event.set()
print("Event has been set.")

# Wait for the threads to finish
for t in threads:
    t.join()


Worker 137194048046656 is waiting for the event.
Worker 137194056439360 is waiting for the event.
Worker 137194668811840 is waiting for the event.
Event has been set.
Worker 137194048046656 has been notified of the event.
Worker 137194668811840 has been notified of the event.
Worker 137194056439360 has been notified of the event.


In [10]:
import threading

def print_message(message):
    print("Timer message:", message)

# Create a timer that prints a message after 5 seconds
timer = threading.Timer(5, print_message, args=("Hello, world!",))

# Start the timer
timer.start()

# Wait for the timer to finish
timer.join()

Timer message: Hello, world!


In [11]:
import threading


def worker():
    print("Worker thread is waiting at the barrier.")
    barrier.wait()
    print("Worker thread has passed the barrier and can proceed.")


# Create a barrier for 3 threads
barrier = threading.Barrier(3)

# Create worker threads
threads = []
for _ in range(3):
    t = threading.Thread(target=worker)
    threads.append(t)
    t.start()

# Wait for the threads to finish
for t in threads:
    t.join()


Worker thread is waiting at the barrier.
Worker thread is waiting at the barrier.
Worker thread is waiting at the barrier.
Worker thread has passed the barrier and can proceed.Worker thread has passed the barrier and can proceed.
Worker thread has passed the barrier and can proceed.

